In [1]:
import sys
sys.path.append('../script/')
import os
from os.path import exists
from datetime import datetime
import json
import gc
from functools import partial

import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import log_loss
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# import optuna

import utils

DEVICE = "cuda"
EPOCHS = 50000
MODELNAME = "LightGBM1019"
if not exists(f"{MODELNAME}/scores"):
    os.makedirs(f"{MODELNAME}/scores")
if not exists(f"{MODELNAME}/weight"):
    os.makedirs(f"{MODELNAME}/weight")
now = datetime.now()
now = str(now)[5:17].replace(" ", "_").replace(":", "")
# writer = SummaryWriter(log_dir=f"{MODELNAME}/tensorboard")

split data

In [2]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

def split_data():
    print("Split data")
    path_fold = "../input/folds/train_folds.csv"
    if not exists(path_fold):
        df = pd.read_csv("../input/lish-moa/train_targets_scored.csv")
        df.loc[:, "kfold"] = -1
        df = df.sample(frac=1).reset_index(drop=True)
        targets = df.drop("sig_id", axis=1).values

        mskf = MultilabelStratifiedKFold(n_splits=5)
        for fold_, (tr_, va_) in enumerate(mskf.split(X=df, y=targets)):
            df.loc[va_, "kfold"] = fold_
        df.to_csv(path_fold, index=False)
        print(f"Created: {path_fold}")
    else:
        print("Skipped: already exists")

preprocess helper functions

In [3]:
def add_pca(df, n_comp_c=15, n_comp_g=50, use_diff=False, drop=False):
    """How to Extract
        with open(f'{MODELNAME}/weight/scaler.npy', 'rb') as f:
            mean_c = np.load(f)
            var_c = np.load(f)
            mean_g = np.load(f)
            var_g = np.load(f)
    """
    c_feats = [col for col in df.columns if col.startswith('c-')]
    g_feats = [col for col in df.columns if col.startswith('g-')]

    c_pca = PCA(n_components=n_comp_c, random_state=42).fit_transform(df[c_feats])
    g_pca = PCA(n_components=n_comp_g, random_state=42).fit_transform(df[g_feats])
    
    cp = pd.DataFrame(c_pca, columns=[f'c_pca{i}' for i in range(n_comp_c)])
    gp = pd.DataFrame(g_pca, columns=[f'g_pca{i}' for i in range(n_comp_g)])

    scaler = StandardScaler()
    cp[cp.columns] = scaler.fit_transform(cp)
    cp[cp.columns] = cp[cp.columns].astype(np.float32)
    mean_c, var_c = scaler.mean_, scaler.var_

    scaler = StandardScaler()
    gp[gp.columns] = scaler.fit_transform(gp)
    gp[gp.columns] = gp[gp.columns].astype(np.float32)
    mean_g, var_g = scaler.mean_, scaler.var_

    with open(f'{MODELNAME}/weight/scaler.npy', 'wb') as f:
        np.save(f, mean_c)
        np.save(f, var_c)
        np.save(f, mean_g)
        np.save(f, var_g)

    if use_diff:
        for i in range(1, cp.shape[1]):
            cp[f'c_pca{i}'] = cp[f'c_pca{i}'] - cp[f'c_pca0']
        for i in range(1, gp.shape[1]):
            gp[f'g_pca{i}'] = gp[f'g_pca{i}'] - gp[f'g_pca0']
    
    df = pd.concat([df, cp, gp], axis=1)
    if drop:
        df.drop(c_feats, axis=1, inplace=True)
        df.drop(g_feats, axis=1, inplace=True)
    return df

## cp_time, cp_doseが同じグループごとに、ターゲットと相関の高い特徴値を選択した列を追加

In [4]:
def add_selective_feats(df, targets):
    
    if exists(f'{MODELNAME}/selected_features.pkl'):
        print('Found existing file. Concatenating...')
        feats = pd.read_pickle(f'{MODELNAME}/selected_features.pkl')
        return pd.concat([df, feats.iloc[:, 1:]], axis=1), feats.columns[1:].values
    
    c_feats = [col for col in df.columns if col.startswith('c-')]
    g_feats = [col for col in df.columns if col.startswith('g-')]
    df['nan'] = np.nan
    print(f'Adding selective features...')
    for t in df['cp_time'].unique():
        for d in df['cp_dose'].unique():
            print(f'(group {t}{d})', end='    ')

            subset = df.query(f'cp_time == {t} and cp_dose == {d}').copy()

            # [c features]
            # Know which feature is the best in the subset
            print('Creating corr matrix for c features...', end='')
            c_corr = np.zeros((len(c_feats), len(targets)))
            for fi, feat in enumerate(c_feats):
                for ti, target_ in enumerate(targets):
                    c_corr[fi,ti] = subset[target_].corr(subset[feat])
            c_corr = pd.DataFrame(c_corr, index=c_feats, columns=targets)

            # [g features]
            # Know which feature is the best in the subset
            print('done. Then matrix for g features...')
            g_corr = np.zeros((len(g_feats), len(targets)))
            for fi, feat in enumerate(g_feats):
                for ti, target in enumerate(targets):
                    g_corr[fi,ti] = subset[target].corr(subset[feat])
            g_corr = pd.DataFrame(g_corr, index=g_feats, columns=targets)
            print('done.')
            
            for i, target in enumerate(targets):
                print(f'Target No.{i:>3} / {len(targets):>3}', end='\r')

                # Insert values of selected feature
                selected_feats = c_corr.idxmax().astype(str)
                df.loc[subset.index, f'cfeat_for_{target}'] = df.loc[subset.index, selected_feats[target]]

                # Insert values of selected feature
                selected_feats = g_corr.idxmax().astype(str)
                df.loc[subset.index, f'gfeat_for_{target}'] = df.loc[subset.index, selected_feats[target]]
            print('\n')
    
    added_features = []
    print('\nCompressing created columns...')
    for i, target in enumerate(targets):
        print(f'Target No.{i:>3} / {len(targets):>3}', end='\r')
        df[f'cfeat_for_{target}'] = df[f'cfeat_for_{target}'].astype(np.float16)
        df[f'gfeat_for_{target}'] = df[f'gfeat_for_{target}'].astype(np.float16)
        added_features.append(f'cfeat_for_{target}')
        added_features.append(f'gfeat_for_{target}')
        
    df = df.drop(['nan'], axis=1)
    return df, added_features

Load and preprocess

In [5]:
def preprocess():
    df = pd.read_csv("../input/lish-moa/train_features.csv")

    # Label encoding
    mapping = {"cp_type":{"trt_cp": 0, "ctl_vehicle":1},
               "cp_time":{24:0, 48:1, 72:2},
               "cp_dose":{"D1":0, "D2":1}}
    for col in ['cp_type', 'cp_time', 'cp_dose']:
        df[col] = df[col].map(mapping[col])

    folds = pd.read_csv("../input/folds/train_folds.csv")

    # Create aux target
    # `nsc_labels` means # of labels found in non-scored train set
    non_scored_df = pd.read_csv("../input/lish-moa/train_targets_nonscored.csv")
    targets_non_scored = non_scored_df.drop("sig_id", axis=1).to_numpy().sum(axis=1)
    non_scored_df.loc[:, "nsc_labels"] = targets_non_scored
    drop_cols = [c for c in non_scored_df.columns if c not in ("nsc_labels", "sig_id")]
    non_scored_df = non_scored_df.drop(drop_cols, axis=1)
    folds = folds.merge(non_scored_df, on="sig_id", how="left")

    # Feature engineering
    # df = add_pca(df)

    targets = folds.drop(["sig_id", "kfold"], axis=1).columns
    features = df.drop("sig_id", axis=1).columns
    df = df.merge(folds, on="sig_id", how="left")

    # Feature engineering (target related)
    df, cols = add_selective_feats(df, targets)   
    features = np.append(features.values, cols)

    return df, features, targets

# split_data()
df, features, targets = preprocess()

Found existing file. Concatenating...


In [6]:
def run_training(df, fold, params, hp_tune=False):
    save_model = False if hp_tune else True
    print(f'\n[Fold No.{fold:>3}]')
    train_df = df[df.kfold != fold].reset_index(drop=True)
    valid_df = df[df.kfold == fold].reset_index(drop=True)

    x_tr = train_df[features].to_numpy()
    x_va = valid_df[features].to_numpy()

    del df
    gc.collect()

    loss_tr, loss_va = [], []
    for i, target in enumerate(targets):
        print(f'Target No.{i:>3} / {len(targets):>3}',end='\r')
        y_tr = train_df[target].to_numpy()
        y_va = valid_df[target].to_numpy()
        dataset_tr = lgb.Dataset(x_tr, label=y_tr)
        dataset_va = lgb.Dataset(x_va, label=y_va, reference=dataset_tr)

        model = lgb.train(params, dataset_tr, EPOCHS, valid_sets=[dataset_tr, dataset_va],
                          verbose_eval=False, early_stopping_rounds=100)

        filename = f"{MODELNAME}/weight/tgt{i}_{now}_fold{fold}.txt"

        preds_tr = model.predict(x_tr, num_iteration=model.best_iteration)
        preds_va = model.predict(x_va, num_iteration=model.best_iteration)

        loss_tr.append(log_loss(y_tr, preds_tr, labels=[0,1]).item())
        loss_va.append(log_loss(y_va, preds_va, labels=[0,1]).item())

        # if save_model:
        #     model.save_model(filename, num_iteration=model.best_iteration)
    
    mean_loss_tr, mean_loss_va = np.mean(loss_tr), np.mean(loss_va)
    print(f"[fold{fold:>2}] {np.mean(mean_loss_tr):.5}, {np.mean(mean_loss_va):.5}")
    with open(f"{MODELNAME}/scores/{now}.txt", "a") as f:
        f.write(f"[fold{fold:>2}] {np.mean(mean_loss_tr):.5}, {np.mean(mean_loss_va):.5}\n")
    print()

    print("\nmodel saved at:", filename)

In [7]:
params = {"metric": 'binary_logloss',
          'objective': 'binary',
          'num_leaves': 491,
          'min_child_weight': 0.03,
          'feature_fraction': 0.3,
          'bagging_fraction': 0.4,
          'min_data_in_leaf': 106,
          'max_depth': -1,
          'learning_rate': 0.01,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "verbose": -1,
          'reg_alpha': 0.4,
          'reg_lambda': 0.6,
          'random_state': 47,
          "force_col_wise": True
         }

In [8]:
%%time
# for fold in range(0,4):
run_training(df, 0, params)


[Fold No.  0]
[fold 0] 0.0073371, 0.017967


model saved at: LightGBM1019/weight/tgt206_10-29_0151_fold0.txt
Wall time: 24min 24s


In [9]:
def show_importance(model, features, now, filename):
    import matplotlib.pyplot as plt
    importance = pd.DataFrame(model.feature_importance(), index=features, columns=['importance'])
    importance = importance.sort_values(by=['importance'], ascending=False)
    plt.figure(figsize=(15,80))
    plt.title(f'feature_importance of experiment {now}')
    plt.tight_layout()
    barlist = plt.barh(importance['importance'].index, importance['importance'].values, height=.9, edgecolor='white')
    for v, bar in zip(importance['importance'].values, barlist):
        v_ = np.clip(v / 256, 0, 1)
        bar.set_color((v_, .5, (1-v_)))
    plt.savefig(filename)
# show_importance(model, features, now, f'{MODELNAME}/{now}_importance.png')

In [10]:
with open(f'{MODELNAME}/{now}_params.json', 'w') as f:
    json.dump(params, f, indent=4)

In [11]:
gc.collect()

769

---
Hyper parameter optimization

In [12]:
def objective(trial):
    params = {
        "nn_params": {"dropout": trial.suggest_uniform("dropout", 0.1, 0.8), 
                      "num_layers": trial.suggest_int("num_layers", 1, 7),
                      "hidden_size": trial.suggest_int("hidden_size", 16, 2048),
                      "activation": trial.suggest_categorical("activation", ["relu", "prelu"]),
                      "batchnorm": trial.suggest_categorical("batchnorm", [True, False])},
        "optimizer": trial.suggest_categorical("optimizer", ["SGD", "Adam"]),
        "optim_params": {"lr": trial.suggest_loguniform("lr", 1e-6, 1e-3)},
        "scheduler": "ReduceLROnPlateau",
        "scdl_params": {"threshold": 0.00001},
    }
    loss_all = []
    for fold_ in range(4):
        loss_tmp = run_training(df, fold, params, save_model=False)
        loss_all.append(loss_tmp)
    return np.mean(loss_all)